In [8]:
import pandas as pd
import numpy as np

In [9]:
def process_seasontickets(seasontickets_path, sf_path):
    # Load all_tickets CSV
    seasontickets = pd.read_csv(seasontickets_path)

    # Rename columns
    seasontickets = seasontickets.rename(columns={
        'Product Id': 'Product ID',
        'Product': 'Asset Name',
        'Id': 'Roboticket ID',
        'Price area': 'Price Area Name',
        'Ticket price types': 'Price Name',
        'Base price': 'Base Price',
        'Area': 'Sector',
        'Type': 'Operation Name',
        'Date.1': 'Product Date',
        'User Id': 'Owner User ID',
        'assign using  ID number': 'Owner Identifier'
    })

    # Add missing columns
    new_columns = [
        'Season ID', 'From', 'To', 'Stadium ID', 'Product Type', 'Product Name', 'Operation', 'Return Type', 'Return ID',
        'Platform', 'Entrance Text', 'Seat ID', 'Price Modifier Value', 'Access Token', 'Ticket ID', 'Transaction Owner',
        'Description', 'Account', 'Account ID', 'Contact', 'Product', 'Purchaser Account',
        'Return Asset', 'Status', 'Is Ticket', 'Is Season Ticket', 'Created By', 'Last Modified By'
    ]
    for col in new_columns:
        if col not in seasontickets.columns:
            seasontickets[col] = None

    # Reorder columns
    desired_columns = [
        'Product ID', 'Season ID', 'Asset Name', 'Roboticket ID', 'Price', 'Price Name', 'Price Area Name', 'Base Price', 'From', 'To',
        'Stadium ID', 'Sector', 'Row', 'Number', 'Product Type', 'Operation Name', 'Product Name', 'Product Date',
        'Operation', 'Return Type', 'Return ID', 'Owner User ID', 'Owner Identifier', 'Platform', 'Entrance Text',
        'Seat ID', 'Price Modifier Value', 'Access Token', 'Ticket ID', 'Transaction Owner', 'Purchaser Account', 'Stadium',
        'Description', 'Account', 'Account ID', 'Contact', 'Product', 'Transaction', 'Return Asset', 'Status', 'Is Ticket',
        'Is Season Ticket', 'Created By', 'Last Modified By'
    ]
    for col in desired_columns:
        if col not in seasontickets.columns:
            seasontickets[col] = None
    seasontickets = seasontickets[desired_columns]

    # Set default values
    seasontickets['Season ID'] = 25
    seasontickets['Product Name'] = seasontickets['Asset Name']
    seasontickets['Product Type'] = 'Match'
    seasontickets['Operation Name'] = 'Sale'
    seasontickets['Purchaser Account'] = seasontickets['Owner User ID']
    seasontickets['Is Ticket'] = False
    seasontickets['Is Season Ticket'] = True
    seasontickets['Stadium ID'] = np.where(
        seasontickets['Stadium'] == 'פיס ארנה', 1, seasontickets['Stadium ID']
    )

    # Map Operation Name to Operation Code
    conditions = [
        seasontickets['Operation Name'] == 'Sale',
        seasontickets['Operation Name'] == 'Return'
    ]
    choices = [1, 2]
    seasontickets['Operation'] = np.select(conditions, choices, default=seasontickets['Operation'])

    # Load SF data
    sf = pd.read_csv(sf_path)
    seasontickets_ids = seasontickets[['Owner Identifier', 'Account ID']].copy()
    merged_ids = seasontickets_ids.merge(
        sf[['id__c', 'AccountId']].astype(str).apply(lambda x: x.str.strip()),
        how='left',
        left_on='Owner Identifier',
        right_on='id__c'
    )
    merged_ids['Account ID'] = merged_ids['Account ID'].fillna(merged_ids['AccountId'])
    merged_ids.drop(columns=['id__c', 'AccountId'], inplace=True)
    seasontickets['Account ID'] = merged_ids['Account ID']

    return seasontickets


In [10]:
df = process_seasontickets(
    seasontickets_path='seasonticket2526.csv',
    sf_path='extract.csv'
)

In [11]:
df

,Product ID,Season ID,Asset Name,Roboticket ID,Price,Price Name,Price Area Name,Base Price,From,To,...,Account ID,Contact,Product,Transaction,Return Asset,Status,Is Ticket,Is Season Ticket,Created By,Last Modified By
0,194,25,מנוי לעונת 2025/2026,900184,0,Complementary,B,0,None,None,...,001Hu00003AEyMEIA1,None,None,3746936,None,Active,False,True,None,None
1,194,25,מנוי לעונת 2025/2026,900183,0,Complementary,F,0,None,None,...,001Hu00003AEyMEIA1,None,None,3746935,None,Active,False,True,None,None
2,194,25,מנוי לעונת 2025/2026,900182,0,Complementary,Silver,0,None,None,...,001Hu00003AEyMEIA1,None,None,3746934,None,Active,False,True,None,None
3,194,25,מנוי לעונת 2025/2026,900181,0,Complementary,Gold,0,None,None,...,001Hu00003AEyMEIA1,None,None,3746933,None,Active,False,True,None,None
4,194,25,מנוי לעונת 2025/2026,900178,0,Complementary,B,0,None,None,...,001Hu00003AEyMEIA1,None,None,3746930,None,Active,False,True,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2352,194,25,מנוי לעונת 2025/2026,884963,-1040,Playoff finals - Adult,C,0,None,None,...,001Hu00003AEvukIAD,None,None,3722669,None,Canceled,False,True,None,None
2353,194,25,מנוי לעונת 2025/2026,884962,-1300,Playoff finals - Adult,C,0,None,None,...,001Hu00003AEvw0IAD,None,None,3722669,None,Canceled,False,True,None,None
2354,194,25,מנוי לעונת 2025/2026,884961,-1300,Playoff finals - Adult,C,0,None,None,...,001Hu00003KorXlIAJ,None,None,3722669,None,Canceled,False,True,None,None
2355,194,25,מנוי לעונת 2025/2026,881315,-1680,Playoff finals - Adult,A,0,None,None,...,001Hu00003AEvLNIA1,None,None,3720769,None,Canceled,False,True,None,None


In [12]:
df.iloc[90]

Product ID                                 194
Season ID                                   25
Asset Name                מנוי לעונת 2025/2026
Roboticket ID                           899490
Price                                      680
Price Name              Playoff finals - Adult
Price Area Name                              D
Base Price                                 850
From                                      None
To                                        None
Stadium ID                                None
Sector                                 12 אולם
Row                                          2
Number                                      30
Product Type                             Match
Operation Name                            Sale
Product Name              מנוי לעונת 2025/2026
Product Date                        2025-06-10
Operation                                    1
Return Type                               None
Return ID                                 None
Owner User ID

In [13]:
df.to_excel('seasontickets2526.xlsx')